<a href="https://colab.research.google.com/github/MiioCh/Python/blob/main/TrabalhoDesenvolvimentoEstoque.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [63]:
import sqlite3
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sys

#Inicialização do banco de dados
conn = sqlite3.connect('database.db')
cur = conn.cursor()

#Classes instanciadas no começo para ultilização ao longo do código
cadastro = CadastroProdutos()
atualizacao = AtualizacaoProdutos()
remover = RemoverProdutos()
relatorio = RelatorioProdutos()
verificador = VerificarSolucao()
verificador.gerar_tabela()

#Variaveis para tabela verdade
P = True
E = True
L = True
R = True

#Classe de usuários e apresentação do menu, apresenta o menu com base na nivel do usuário protegido por login e senha, Gerente tem acesso a todas as funcioalidades, o estoquista tem acesso apenas a algumas funcioanlidade
#o usuário comum apenas tem acesso a lista de itens cadastrados
class NivelUsuario:
    def __init__(self, db_name='database.db'):
        self.conn = sqlite3.connect(db_name)
        self.cur = self.conn.cursor()

    def verificar_requisitos(self, P, E, L, R):
        if P and E and L and R:
            return True
        else:
            return False
#Usuário: GerentedeSetor Senha = Gerente1
#usuario == Estoquista and senha = Estoquista1
#usuario == Usuario and senha = Usuario1
    def autenticar_usuario(self):
        usuario = input("Digite seu usuário: ")
        senha = input("Digite sua senha: ")

        if usuario == "GerentedeSetor" and senha == "Gerente1":
            print("Você tem acesso a todas as funcionalidades do sistema")
            self.exibir_menu('GerentedeSetor')
        elif usuario == "Estoquista" and senha == "Estoquista1":
            print("Você tem acesso ao sistema para dar entrada em produtos e validar notas fiscais")
            self.exibir_menu('Estoquista')
        elif usuario == "Usuario" and senha == "Usuario1":
            print("Você tem acesso aos relatórios")
            self.exibir_menu('Usuario')
        else:
            print("Usuário ou senha incorretos. Acesso negado.")
#Ultilizado para criar a tabela verdade
    combinacoes = [
        (True, True, True, True),
        (True, True, True, False),
        (True, True, False, True),
        (True, True, False, False),
        (True, False, True, True),
        (True, False, True, False),
        (True, False, False, True),
        (True, False, False, False),
        (False, True, True, True),
        (False, True, True, False),
        (False, True, False, True),
        (False, True, False, False),
        (False, False, True, True),
        (False, False, True, False),
        (False, False, False, True),
        (False, False, False, False)
    ]

    def verificar_solucao_completa(self, P, E, L, R):
        return P and E and L and R
#Funcioanlidade que exibe o menu
    def exibir_menu(self, nivel):
        if nivel == "GerentedeSetor":
            print("Digite 1 para cadastrar um produto")
            print("Digite 2 para atualizar um produto")
            print("Digite 3 para remover um produto")
            print("Digite 4 para verificar a lista de itens cadastrados")
            print("Digite 5 para gerar um relatório de produtos em baixa")
            print("Digite 6 para verificar se o programa está funcional")
            print("Digite 7 para sair do sistema")
            print("Digite 8 para visualizar todas as movimentações no estoque")  # Nova opção

        elif nivel == "Estoquista":
            print("Digite 1 para cadastrar um produto")
            print("Digite 2 para atualizar um produto")
            print("Digite 3 para remover um produto")
            print("Digite 6 para sair do sistema")

        elif nivel == "Usuario":
            print("Digite 4 para verificar a lista de itens cadastrados")
            print("Digite 6 para sair do sistema")

        opcao = int(input("Escolha uma opção: "))
        self.processar_opcao(nivel, opcao)

    def processar_opcao(self, nivel, opcao):
        if opcao == 8 and nivel == "GerentedeSetor":
            movimentacao = MovimentacaoProdutos()
            movimentacao.exibir_movimentacoes()

            opcao = int(input("Escolha uma opção: "))
            self.processar_opcao(nivel, opcao)

    def processar_opcao(self, nivel, opcao):
        if opcao == 8 and nivel == "GerentedeSetor":
            movimentacao = MovimentacaoProdutos()
            movimentacao.exibir_movimentacoes()

        opcao = int(input("Escolha uma opção: "))
        self.processar_opcao(nivel, opcao)

#Condicionais para verificar o nível de acesso e apresentar o menu
    def processar_opcao(self, nivel, opcao):
        if opcao == 1:
            adicionar_produto = input("Informe nome, categoria, quantidade, preço e localização (separados por vírgula): ")
            dados = adicionar_produto.split(",")
            if len(dados) == 5:
                nome, categoria, quantidade, preco, locacao = dados
                cadastro.adicionar_produto(nome, categoria, int(quantidade), float(preco), locacao)
            else:
                print("Erro: Certifique-se de inserir todas as informações corretamente.")

        if opcao == 2:
            atualizar_produto = input("Informe nome, quantidade, preço e localização (separados por vírgula): ")
            dados = atualizar_produto.split(",")
            if len(dados) == 4:
                nome, quantidade, preco, locacao = dados
                atualizacao.atualizar_produto(nome, int(quantidade), float(preco), locacao)

        if opcao == 3:
            df_produtos = pd.read_sql_query("SELECT * from products", cadastro.conn)
            display(df_produtos)
            remover_produto = input("Informe o nome do produto que deseja remover: ")
            remover.remover_produtos(remover_produto)

        if opcao == 4:
            df_produtos = pd.read_sql_query("SELECT * from products", cadastro.conn)
            display(df_produtos)

        if opcao == 5:
            limite = int(5)
            relatorio.produtos_em_baixa(limite=limite)

        if opcao == 6:
            if nivel == "GerentedeSetor":
                print("Tabela para verificar todas as funcionalidades do sistema: ")
                for combinacao in self.combinacoes:
                    P, E, L, R = combinacao
                    solucao_completa = self.verificar_solucao_completa(P, E, L, R)
                    print(f"{P:<5}{E:<5}{L:<5}{R:<5}{'T' if solucao_completa else 'F'}")
            else:
                print("Apenas o Gerente de Setor tem permissão para verificar se o sistema está funcional.")

        if opcao == 7:
            print("Encerrando o programa. Até logo!")
            sys.exit()

        if opcao == 8:  # Nova opção para o Gerente
            if nivel == "GerentedeSetor":
                self.exibir_movimentacoes()
            else:
                print("Apenas o Gerente de Setor pode ver as movimentações.")

    def exibir_movimentacoes(self):
        try:
            self.cur.execute('''SELECT * FROM movimentacoes''')
            movimentacoes = self.cur.fetchall()

            if movimentacoes:
                print("\nMovimentações realizadas no estoque:")
                for movimentacao in movimentacoes:
                    print(f"ID: {movimentacao[0]}, Produto: {movimentacao[1]}, Tipo: {movimentacao[2]}, "
                          f"Quantidade: {movimentacao[3]}, Preço: {movimentacao[4]}, Localização: {movimentacao[5]}, "
                          f"Data: {movimentacao[6]}, Usuário Responsável: {movimentacao[7]}")
            else:
                print("\nNão há movimentações registradas no estoque.")
        except sqlite3.Error as e:
            print(f'Erro ao recuperar movimentações: {e}')

# Instância de NivelUsuario para autenticação
nivelusuario = NivelUsuario()
nivelusuario.autenticar_usuario()
nivelusuario = NivelUsuario()

#Classe de cadastro de produtos, cria uma tabela se não existir e pede os dados ao usuário
class CadastroProdutos:
    def __init__(self, db_name='database.db'):
        self.conn = sqlite3.connect(db_name)
        self.cur = self.conn.cursor()
        self.criar_tabela()

    def criar_tabela(self):
        self.cur.execute('''
            CREATE TABLE IF NOT EXISTS products (
                id INTEGER PRIMARY KEY,
                Nome TEXT NOT NULL,
                Categoria TEXT NOT NULL,
                Quantidade INTEGER NOT NULL,
                Preço REAL NOT NULL,
                Locação TEXT NOT NULL
            )
        ''')
        self.conn.commit()

    def adicionar_produto(self, nome, categoria, quantidade, preco, locacao):
        try:
            self.cur.execute('''
                INSERT INTO products (Nome, Categoria, Quantidade, Preço, Locação)
                VALUES (?, ?, ?, ?, ?)
            ''', (nome, categoria, quantidade, preco, locacao))
            self.conn.commit()
            print(f'O produto {nome} foi adicionado com sucesso!')
        except sqlite3.Error as e:
            print(f'Erro ao adicionar o produto: {e}')

    def fechar_conexao(self):
        self.conn.close()

#Classe de atualização de produtos
class AtualizacaoProdutos:
    def __init__(self, db_name='database.db'):
        self.conn = sqlite3.connect(db_name)
        self.cur = self.conn.cursor()

    def atualizar_produto(self, nome, quantidade, preco, locacao):
        try:
            self.cur.execute('''
                UPDATE products
                SET Quantidade = ?, Preço = ?, Locação = ?
                WHERE Nome = ?
            ''', (quantidade, preco, locacao, nome))
            self.conn.commit()
            print(f'O produto {nome} foi atualizado com sucesso!')
        except sqlite3.Error as e:
            print(f'Erro ao atualizar o produto: {e}')

    def fechar_conexao(self):
        self.conn.close()

#Classe para remover produtos
class RemoverProdutos:
    def __init__(self, db_name='database.db'):
        self.conn = sqlite3.connect(db_name)
        self.cur = self.conn.cursor()

    def remover_produtos(self, nome):
        try:
            self.cur.execute('''
                DELETE FROM products
                WHERE Nome = ?
            ''', (nome,))
            self.conn.commit()
            print(f'O produto {nome} foi removido com sucesso!')
        except sqlite3.Error as e:
            print(f'Erro ao remover o produto: {e}')

    def fechar_conexao(self):
        self.conn.close()


#Classe para gerar relatórios, imprime automaticamente um aviso caso algum produto esteja em baixo, preferencialmente menos de 5
class RelatorioProdutos:
    def __init__(self, db_name='database.db'):
        self.conn = sqlite3.connect(db_name)
        self.cur = self.conn.cursor()

    def produtos_em_baixa(self, limite=5):
        try:
            self.cur.execute('''
                SELECT * FROM products
                WHERE Quantidade < ?
            ''', (limite,))
            produtos_baixa = self.cur.fetchall()

            if produtos_baixa:
                df_produtos_baixa = pd.DataFrame(produtos_baixa, columns=['ID', 'Nome', 'Categoria', 'Quantidade', 'Preço', 'Locação'])
                print("\nProdutos com baixa quantidade em estoque:")
                print(df_produtos_baixa)
            else:
                print("\nTodos os produtos estão em quantidade suficiente.")

        except sqlite3.Error as e:
            print(f'Erro ao gerar relatório: {e}')

    def fechar_conexao(self):
        self.conn.close()

#Classe para verificar quais movimentações são feitas no estoque
class MovimentacaoProdutos:
    def __init__(self, db_name='database.db'):
        self.conn = sqlite3.connect(db_name)
        self.cur = self.conn.cursor()
        self.criar_tabela_movimentacao()

    def criar_tabela_movimentacao(self):
        try:
            self.cur.execute('''
                CREATE TABLE IF NOT EXISTS movimentacoes (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    produto_nome TEXT NOT NULL,
                    tipo_operacao TEXT NOT NULL,  -- Inserção, Retirada, Atualização
                    quantidade INTEGER NOT NULL,
                    preco REAL NOT NULL,
                    locacao TEXT NOT NULL,
                    data_movimentacao TEXT NOT NULL,
                    usuario_responsavel TEXT NOT NULL
                )
            ''')
            self.conn.commit()
        except sqlite3.Error as e:
            print(f"Erro ao criar tabela de movimentações: {e}")

    def registrar_movimentacao(self, nome, tipo_operacao, quantidade, preco, locacao, usuario_responsavel):
        from datetime import datetime


        data_movimentacao = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

        try:

            self.cur.execute('''
                INSERT INTO movimentacoes (produto_nome, tipo_operacao, quantidade, preco, locacao, data_movimentacao, usuario_responsavel)
                VALUES (?, ?, ?, ?, ?, ?, ?)
            ''', (nome, tipo_operacao, quantidade, preco, locacao, data_movimentacao, usuario_responsavel))

            self.conn.commit()
        except sqlite3.Error as e:
            print(f"Erro ao registrar movimentação: {e}")

    def inserir_produto(self, nome, quantidade, preco, locacao, usuario_responsavel):
        tipo_operacao = "Inserção"
        self.registrar_movimentacao(nome, tipo_operacao, quantidade, preco, locacao, usuario_responsavel)

    def atualizar_produto(self, nome, quantidade, preco, locacao, usuario_responsavel):
        tipo_operacao = "Atualização"
        self.registrar_movimentacao(nome, tipo_operacao, quantidade, preco, locacao, usuario_responsavel)

    def remover_produto(self, nome, quantidade, preco, locacao, usuario_responsavel):
        tipo_operacao = "Retirada"
        self.registrar_movimentacao(nome, tipo_operacao, quantidade, preco, locacao, usuario_responsavel)

    def exibir_movimentacoes(self):
        try:
            self.cur.execute('SELECT * FROM movimentacoes')
            movimentacoes = self.cur.fetchall()

            if movimentacoes:
                print("\nMovimentações no estoque:")
                for mov in movimentacoes:
                    print(mov)
            else:
                print("\nNenhuma movimentação registrada.")
        except sqlite3.Error as e:
            print(f"Erro ao recuperar movimentações: {e}")




produtos = {}


conn.commit()


conn.close()

Digite seu usuário: GerentedeSetor
Digite sua senha: Gerente1
Você tem acesso a todas as funcionalidades do sistema
Digite 1 para cadastrar um produto
Digite 2 para atualizar um produto
Digite 3 para remover um produto
Digite 4 para verificar a lista de itens cadastrados
Digite 5 para gerar um relatório de produtos em baixa
Digite 6 para verificar se o programa está funcional
Digite 7 para sair do sistema
Digite 8 para visualizar todas as movimentações no estoque
Escolha uma opção: 6
Tabela para verificar todas as funcionalidades do sistema: 
1    1    1    1    T
1    1    1    0    F
1    1    0    1    F
1    1    0    0    F
1    0    1    1    F
1    0    1    0    F
1    0    0    1    F
1    0    0    0    F
0    1    1    1    F
0    1    1    0    F
0    1    0    1    F
0    1    0    0    F
0    0    1    1    F
0    0    1    0    F
0    0    0    1    F
0    0    0    0    F
